In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import pprint
import os

In [2]:
# stations data path
file_path = '../Resources/stationslist_with_Routetrip.csv'

# Read CSV and convert to stations_df
stations_df=pd.read_csv(file_path)

# display df
stations_df

,latitude,longitude,stations_Route_trip,stationCode
0,-29.063612,114.997161,4,AN001
1,-34.270827,118.268523,2,AM001
2,-32.125480,118.004060,0,SH002
3,-30.338049,115.539491,4,BA
4,-33.796200,116.063980,2,BP001
...,...,...,...,...
219,-34.225653,116.017767,2,MA003
220,-31.915622,119.256055,3,YS
221,-31.835878,116.921077,0,YE001
222,-28.337630,114.989800,4,YU001


In [3]:
# rain data path
file_path = '../Resources/raindata_with_perdiction.csv'

# Read CSV and convert to rain_df
rain_df=pd.read_csv(file_path)

# # Convert date into number
rain_df['Date'] = pd.to_datetime(rain_df[['Year', 'Month', 'Day']], errors='coerce')


rain_df

,Year,Month,Day,stationCode,stationName,relativeHumidity,airTemperature_avg,wind_avg_speed,Actual_Rain,Predicted_Rain,Date
0,2023.0,1.0,1.0,AN001,Allanooka,66.1,21.0,17.93,0.0,0.0,2023-01-01
1,2023.0,1.0,2.0,AN001,Allanooka,52.2,25.2,16.49,0.0,0.0,2023-01-02
2,2023.0,1.0,3.0,AN001,Allanooka,19.1,32.2,24.77,1.2,1.0,2023-01-03
3,2023.0,1.0,4.0,AN001,Allanooka,22.9,32.1,21.67,0.0,0.0,2023-01-04
4,2023.0,1.0,5.0,AN001,Allanooka,17.8,33.3,18.12,0.0,0.0,2023-01-05
...,...,...,...,...,...,...,...,...,...,...,...
47162,2023.0,8.0,9.0,YU002,Yuna NE,74.2,14.2,5.72,0.0,2.0,2023-08-09
47163,2023.0,8.0,10.0,YU002,Yuna NE,72.1,14.8,5.71,0.0,0.0,2023-08-10
47164,2023.0,8.0,11.0,YU002,Yuna NE,72.8,16.8,6.62,0.0,0.0,2023-08-11
47165,2023.0,8.0,12.0,YU002,Yuna NE,75.7,15.9,7.51,0.0,0.0,2023-08-12


In [4]:
# find the unique values in the 'stationCode' column of the rain_df
stations= rain_df['stationCode'].unique()

In [5]:
# Initialize a dictionary to store dictionaries for each station
all_stations = {}

# Iterate through each station
for station in stations:
    # Filter rows based on the 'station' column for the rain data
    filtered_row = rain_df[rain_df['stationCode'] == station]
    
    # Initialize empty arrays to store all rain data for each station for different dates
    rain_array = []
    prain_array = []
    year_array = []
    month_array = []
    day_array = []
    date_array = []

    # Iterate through the DataFrame and populate the rain amount arrays
    for i in range(len(filtered_row)):
        rain = filtered_row.iloc[i]['Actual_Rain']
        prain = filtered_row.iloc[i]['Predicted_Rain']
        year = filtered_row.iloc[i]['Year']
        month = filtered_row.iloc[i]['Month']
        day = filtered_row.iloc[i]['Day']
        date = filtered_row.iloc[i]['Date']

        rain_array.append(rain)
        prain_array.append(prain)
        year_array.append(year)
        month_array.append(month)
        day_array.append(day)
        date_array.append(date)

    # Create a dictionary with the desired structure for the current rain station
    station_dict = {
        "stationCode": station,
        "lat": stations_df[stations_df['stationCode'] == station]['latitude'].values[0],
        "lon": stations_df[stations_df['stationCode'] == station]['longitude'].values[0],
        "Routetrip": stations_df[stations_df['stationCode'] == station]['stations_Route_trip'].values[0],
        "Year": year_array,
        "Month": month_array,
        "Day": day_array,
        "Date": date_array,
        "Rain": rain_array,
        "PerdictRain": prain_array
    }

    # Update the all_stations dictionary with the station_dict
    all_stations[station] = station_dict

# Display the result
# all_stations
    

In [8]:
# Function to convert NumPy integers to regular Python integers
def convert_to_serializable(item):
    if isinstance(item, (np.integer, np.int64)):
        return int(item)
    elif isinstance(item, pd.Timestamp):
        return item.strftime('%Y-%m-%d')
    raise TypeError(f"Type {type(item)} not serializable")

# Convert dictionary to JSON string, using the custom converter
json_data = json.dumps(all_stations, indent=2, default=convert_to_serializable)

# Define the JavaScript variable assignment
js_content = f"var alldata = {json_data};"

# Write the JavaScript content to a file
with open('../Map/static/data/data.js', 'w') as js_file:
    js_file.write(js_content)